<a href="https://colab.research.google.com/github/omarhisham007/Deep-Learning/blob/main/emotion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.7.0.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.6/489.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 40.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2


In [3]:
%cd /content/drive/MyDrive/content/emotion_input_output
! ls

/content/drive/MyDrive/content/emotion_input_output
input  output


In [4]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd

from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

In [5]:
dataset_folder='./input/CK+48'
sub_folders=os.listdir(dataset_folder)

sub_folders

['sadness', 'surprise', 'fear', 'anger', 'disgust', 'happy', 'contempt']

In [6]:
i=0
last=[]
images=[]
labels=[]
temp = sub_folders

# reading folders in the main dataset folder, one at a time
for sub_folder in sub_folders:
  sub_folder_index = temp.index(sub_folder)
  label = sub_folder_index

  # Define labels basis use case. We are using positive:0, negative:1, neutral:2 
  if  label in [4, 6]: 
    new_label=0         
  elif label in [0,5]:  
    new_label=1       
  else:           
    new_label=2   
  
  
  path = dataset_folder+'/'+sub_folder
  sub_folder_images= os.listdir(path)
  
  # reading images in the sub folder, one at a time
  for image in sub_folder_images:
    image_path = path+'/'+image
    print(image_path+"\t"+str(new_label))
    
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image= cv2.resize(image,(48,48))
    images.append(image)
    labels.append(new_label)
    i+=1
  last.append(i)

./input/CK+48/sadness/S504_006_00000017.png	1
./input/CK+48/sadness/S506_006_00000040.png	1
./input/CK+48/sadness/S505_006_00000018.png	1
./input/CK+48/sadness/S501_006_00000039.png	1
./input/CK+48/sadness/S137_005_00000025.png	1
./input/CK+48/sadness/S501_006_00000040.png	1
./input/CK+48/sadness/S138_007_00000010.png	1
./input/CK+48/sadness/S503_006_00000020.png	1
./input/CK+48/sadness/S504_006_00000018.png	1
./input/CK+48/sadness/S137_005_00000026.png	1
./input/CK+48/sadness/S505_006_00000019.png	1
./input/CK+48/sadness/S138_007_00000011.png	1
./input/CK+48/sadness/S505_006_00000017.png	1
./input/CK+48/sadness/S506_006_00000041.png	1
./input/CK+48/sadness/S138_007_00000009.png	1
./input/CK+48/sadness/S503_006_00000019.png	1
./input/CK+48/sadness/S506_006_00000042.png	1
./input/CK+48/sadness/S503_006_00000018.png	1
./input/CK+48/sadness/S081_002_00000024.png	1
./input/CK+48/sadness/S115_004_00000017.png	1
./input/CK+48/sadness/S125_001_00000012.png	1
./input/CK+48/sadness/S046_001_000

In [7]:
images_x = np.array(images)
labels_y = np.array(labels)

images_x = images_x/255

In [8]:
images_x.shape

(991, 48, 48)

In [9]:
# encoding the labels
num_of_classes = 3
labels_y_encoded = tf.keras.utils.to_categorical(labels_y,num_classes=num_of_classes)

In [10]:
X_train, X_test, Y_train, Y_test= train_test_split(images_x, labels_y_encoded,test_size=0.25, random_state=10)

In [11]:
len(X_train)

743

In [12]:
len(X_test)

248

In [13]:
input = Input(shape = (48,48,1))

conv1 = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size = (2,2)) (conv1)

conv2 = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size = (2,2)) (conv2)

conv3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size = (2,2)) (conv3)

conv4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size = (2,2)) (conv4)

flatten = Flatten()(pool4)

dense_1 = Dense(128,activation='relu')(flatten)

drop_1 = Dropout(0.2)(dense_1)

output = Dense(3,activation="sigmoid")(drop_1)

In [14]:
model = Model(inputs=input,outputs=output)
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 dropout (Dropout)           (None, 48, 48, 32)        0         
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                             

In [15]:
fle_s='./output/emotion_model.h5'
checkpointer = ModelCheckpoint(fle_s, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]

In [16]:
save = model.fit(X_train,Y_train,batch_size=32,validation_data=(X_test,Y_test),epochs=30,callbacks=[callback_list])

Epoch 1/30
24/24 [==============================] - ETA: 0s - loss: 1.3230 - accuracy: 0.4307
Epoch 00001: loss improved from inf to 1.32303, saving model to ./output/emotion_model.h5
24/24 [==============================] - 15s 585ms/step - loss: 1.3230 - accuracy: 0.4307 - val_loss: 1.2309 - val_accuracy: 0.4798
Epoch 2/30
24/24 [==============================] - ETA: 0s - loss: 1.1988 - accuracy: 0.4576
Epoch 00002: loss improved from 1.32303 to 1.19877, saving model to ./output/emotion_model.h5
24/24 [==============================] - 8s 330ms/step - loss: 1.1988 - accuracy: 0.4576 - val_loss: 1.1686 - val_accuracy: 0.4798
Epoch 3/30
24/24 [==============================] - ETA: 0s - loss: 1.1515 - accuracy: 0.4576
Epoch 00003: loss improved from 1.19877 to 1.15149, saving model to ./output/emotion_model.h5
24/24 [==============================] - 7s 310ms/step - loss: 1.1515 - accuracy: 0.4576 - val_loss: 1.1222 - val_accuracy: 0.4798
Epoch 4/30
24/24 [============================